In [62]:
import numpy as np
import pandas as pd
import os
import glob
import codecs
import scispacy
import spacy
import sklearn
import warnings 

from collections import defaultdict, Counter
from sklearn.model_selection import KFold
# from negspacy.negation import Negex
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import defaultdict
from transformers import BertTokenizer
from nltk.corpus import stopwords

In [63]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [64]:

  
warnings.filterwarnings(action = 'ignore') 
  
# print(model1.similarity("Hospice", "Immunology"))                               
# nlp0 = spacy.load("en_core_web_sm")
# negex = Negex(nlp0, language = "en_clinical")
# nlp0.add_pipe(negex, last=True)
# doc0 = nlp0("he is not an emotional eater")
import string

#string punctuation
punct = set(string.punctuation)
# for e in doc0.ents:
# 	print(e.text, e._.negex)

# Using scispacy with "en_core_sci_sm":
nlp = spacy.load("en_core_sci_sm")

# # Using scispacy with "en_core_sci_lg":
# nlp = spacy.load("en_core_sci_lg")

# # Using scispacy with "en_ner_bc5cdr_md:
# nlp = spacy.load("en_ner_bc5cdr_md")


def tokenize_doc(text):
    bow = defaultdict(float)
    temp = []
    doc = nlp(text)
    doc = doc.ents
    lowered_tokens = map(lambda t: str(t).lower(), doc)

    for token in lowered_tokens:
        # token = token.split(".,")
        # if (len(token) > 1):
        #     for j in token:
        #         j = j.split(":,")
        #         for i in j:
        #             bow[i] += 1.0
        #             temp.append(i)
        # else:
        #     for i in token:
        #         bow[i] += 1.0
        #         temp.append(i)

        temp.append(token)
    return temp

def tokenize_doc2(text):
    t = BertTokenizer.from_pretrained("bert-large-uncased-vocab.txt", lowercase=True)
    warnings.filterwarnings('ignore')
    output = t.tokenize(text)
    temp = []
    words = [w for w in output if w not in punct]
    for token in words:
        temp.append(token)
    return temp

In [65]:
import warnings
warnings.filterwarnings('ignore')

In [66]:
data = pd.read_csv("mtsamples.csv")

In [67]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]
data = data.drop(temp)

        

In [68]:
data = data.reset_index(drop=True)

In [69]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]

surgery = list(data[data["medical_specialty"] == ' Surgery'].index.values)
consult = list(data[data["medical_specialty"] == ' Consult - History and Phy.'].index.values)
cardiovascular = list(data[data["medical_specialty"] == ' Cardiovascular / Pulmonary'].index.values)
orthopedic = list(data[data["medical_specialty"] == ' Orthopedic'].index.values)
radiology = list(data[data["medical_specialty"] == ' Radiology'].index.values)

general = list(data[data["medical_specialty"] == ' General Medicine'].index.values)
gastroenterology = list(data[data["medical_specialty"] == ' Gastroenterology'].index.values)
neurology = list(data[data["medical_specialty"] == ' Neurology'].index.values)
soap = list(data[data["medical_specialty"] == ' SOAP / Chart / Progress Notes'].index.values)
obstetrics = list(data[data["medical_specialty"] == ' Obstetrics / Gynecology'].index.values)

urology = list(data[data["medical_specialty"] == ' Urology'].index.values)
discharge = list(data[data["medical_specialty"] == ' Discharge Summary'].index.values)
ent = list(data[data["medical_specialty"] == ' ENT - Otolaryngology'].index.values)
neurosurgery = list(data[data["medical_specialty"] == ' Neurosurgery'].index.values)
hematology = list(data[data["medical_specialty"] == ' Hematology - Oncology'].index.values)

ophthalmology = list(data[data["medical_specialty"] == ' Ophthalmology'].index.values)
Nephrology = list(data[data["medical_specialty"] == ' Nephrology'].index.values)


uniq_dict = {}

temp_list = surgery[76:] + consult[76:] + cardiovascular[76:] + orthopedic[76:] + radiology[76:] + general[76:] + gastroenterology[76:] + neurology[76:] + soap[76:] + obstetrics[76:] + urology[76:] + discharge[76:] + ent[76:] + neurosurgery[76:] + hematology[76:] + ophthalmology[76:] + Nephrology[76:]

In [70]:
print(temp_list)

[252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451,

In [71]:
# train_data_temp = pd.read_csv("list.csv")
# temp = [i for i in train_data_temp.index if pd.isna(train_data_temp["transcription"][i])]
# train_data_temp = train_data_temp.drop(temp)

data_b = pd.read_csv("mtsamples_b.csv")
data_p = pd.read_csv("mtsamples_p.csv")
data_t = pd.read_csv("mtsamples_t.csv")
data_j1 = pd.read_csv("mtsamples_j1.csv")
data_j2 = pd.read_csv("mtsamples_j2.csv")
data_copy = pd.read_csv("mtsamples_copy.csv")

temp_b = [i for i in data_b.index if pd.isna(data_b["transcription"][i])]
data_b = data_b.drop(temp_b)

temp_p = [i for i in data_p.index if pd.isna(data_p["transcription"][i])]
data_p = data_p.drop(temp_p)

temp_t = [i for i in data_t.index if pd.isna(data_t["transcription"][i])]
data_t = data_t.drop(temp_t)

temp_j1 = [i for i in data_j1.index if pd.isna(data_j1["transcription"][i])]
data_j1 = data_j1.drop(temp_j1)

temp_j2 = [i for i in data_j2.index if pd.isna(data_j2["transcription"][i])]
data_j2 = data_j2.drop(temp_j2)

temp_copy = [i for i in data_copy.index if pd.isna(data_copy["transcription"][i])]
data_copy = data_copy.drop(temp_copy)

data_b = data_b.reset_index(drop=True)
data_p = data_p.reset_index(drop=True)
data_t = data_t.reset_index(drop=True)
data_j1 = data_j1.reset_index(drop=True)
data_j2 = data_j2.reset_index(drop=True)

data_copy = data_copy.reset_index(drop=True)

In [34]:
# print(train_data_temp['transcription'][0])

# print(data_b['transcription'][0])

In [35]:
# print(tokenize_doc(train_data_temp['transcription'][0])  )
# print(tokenize_doc(data_b['transcription'][0])  )

In [72]:
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

nsvc = NuSVC(nu=0.0001, probability=True)
lsvc = LinearSVC(tol = 1e-5, C = 0.0001, random_state=500000)

In [73]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kf = KFold(n_splits=12, random_state=None, shuffle=False)
# kf.get_n_splits(data_b)
# kf.get_n_splits(data_p)
# kf.get_n_splits(data_t)
# kf.get_n_splits(data_j1)
# kf.get_n_splits(data_j2)

# Tfidf_vect_b = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect_p = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect_t = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect_j1 = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect_j2 = TfidfVectorizer(tokenizer=tokenize_doc)
Tfidf_vect_copy = TfidfVectorizer(tokenizer=tokenize_doc)

# Tfidf_vect_b.fit(data_b['transcription'])
# Tfidf_vect_p.fit(data_p['transcription'])
# Tfidf_vect_t.fit(data_t['transcription'])
# Tfidf_vect_j1.fit(data_j1['transcription'])
# Tfidf_vect_j2.fit(data_j2['transcription'])
Tfidf_vect_copy.fit(data_copy['transcription'])



TfidfVectorizer(tokenizer=<function tokenize_doc at 0x0000027439C51F70>)

In [52]:
# kf.get_n_splits(data_b)

12

In [74]:
#copy
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data_b):
    x_train = []
    y_train = []

    x_train, x_test = data_copy['transcription'][train_index], data_copy['transcription'][test_index]
    y_train, y_test = data_copy['medical_specialty'][train_index], data_copy['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
    lsvc.fit(Train_X_Tfidf,y_train) 
    
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())
    print(accuracy_score(y_test, predictions_nuSVM)*100)
    
print(total/count)
print(res)

              precision    recall  f1-score   support

   medicines    1.00000   0.40097   0.57241       414
     surgery    0.00000   0.00000   0.00000         0

    accuracy                        0.40097       414
   macro avg    0.50000   0.20048   0.28621       414
weighted avg    1.00000   0.40097   0.57241       414

40.09661835748793
              precision    recall  f1-score   support

   medicines    0.00000   0.00000   0.00000     414.0
     surgery    0.00000   0.00000   0.00000       0.0

    accuracy                        0.00000     414.0
   macro avg    0.00000   0.00000   0.00000     414.0
weighted avg    0.00000   0.00000   0.00000     414.0

0.0
              precision    recall  f1-score   support

   medicines    0.00000   0.00000   0.00000     414.0
     surgery    0.00000   0.00000   0.00000       0.0

    accuracy                        0.00000     414.0
   macro avg    0.00000   0.00000   0.00000     414.0
weighted avg    0.00000   0.00000   0.00000     414.

In [53]:
#brian
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data_b):
    x_train = []
    y_train = []

    x_train, x_test = data_b['transcription'][train_index], data_b['transcription'][test_index]
    y_train, y_test = data_b['medical_specialty'][train_index], data_b['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
    lsvc.fit(Train_X_Tfidf,y_train) 
    
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())
    print(accuracy_score(y_test, predictions_nuSVM)*100)
    
print(total/count)
print(res)

                       precision    recall  f1-score   support

                 bone    0.00000   0.00000   0.00000         0
             medicine    0.00000   0.00000   0.00000         0
      nervous_related    0.00000   0.00000   0.00000         0
ology_head_whole_body    0.00000   0.00000   0.00000         0
ology_upper_extremity    0.00000   0.00000   0.00000         0
                other    1.00000   0.02415   0.04717       414
        surgery_brian    0.00000   0.00000   0.00000         0

             accuracy                        0.02415       414
            macro avg    0.14286   0.00345   0.00674       414
         weighted avg    1.00000   0.02415   0.04717       414

2.4154589371980677
                       precision    recall  f1-score   support

ology_upper_extremity    0.00000   0.00000   0.00000     414.0
        surgery_brian    0.00000   0.00000   0.00000       0.0

             accuracy                        0.00000     414.0
            macro avg    0.0000

In [54]:
kf.get_n_splits(data_p)

12

In [55]:
#patrick
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data_p):
    x_train = []
    y_train = []

    x_train, x_test = data_p['transcription'][train_index], data_p['transcription'][test_index]
    y_train, y_test = data_p['medical_specialty'][train_index], data_p['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
    lsvc.fit(Train_X_Tfidf,y_train) 
    
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())
    print(accuracy_score(y_test, predictions_nuSVM)*100)
    
print(total/count)
print(res)

              precision    recall  f1-score   support

   specialty    1.00000   0.42271   0.59423       414
     surgery    0.00000   0.00000   0.00000         0
   treatment    0.00000   0.00000   0.00000         0

    accuracy                        0.42271       414
   macro avg    0.33333   0.14090   0.19808       414
weighted avg    1.00000   0.42271   0.59423       414

42.270531400966185
              precision    recall  f1-score   support

   specialty    0.00000   0.00000   0.00000     414.0
     surgery    0.00000   0.00000   0.00000       0.0

    accuracy                        0.00000     414.0
   macro avg    0.00000   0.00000   0.00000     414.0
weighted avg    0.00000   0.00000   0.00000     414.0

0.0
              precision    recall  f1-score   support

   specialty    0.00000   0.00000   0.00000     414.0
     surgery    0.00000   0.00000   0.00000       0.0

    accuracy                        0.00000     414.0
   macro avg    0.00000   0.00000   0.00000     414

In [56]:
kf.get_n_splits(data_t)

12

In [57]:
#thai
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data_t):
    x_train = []
    y_train = []

    x_train, x_test = data_t['transcription'][train_index], data_t['transcription'][test_index]
    y_train, y_test = data_t['medical_specialty'][train_index], data_t['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
    lsvc.fit(Train_X_Tfidf,y_train) 
    
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())
    print(accuracy_score(y_test, predictions_nuSVM)*100)
    
print(total/count)
print(res)

                                                                     precision    recall  f1-score   support

                                             bones/teeth/legs/neuro    0.50000   0.02000   0.03846        50
diseases/serious illnesses/allergy/nutrition/general/birth/children    1.00000   0.03306   0.06400       363
                                  intestine/uniary/skin/heart+blood    0.00000   0.00000   0.00000         0
                                                              notes    0.00000   0.00000   0.00000         1
                                                            surgery    0.00000   0.00000   0.00000         0

                                                           accuracy                        0.03140       414
                                                          macro avg    0.30000   0.01061   0.02049       414
                                                       weighted avg    0.93720   0.03140   0.06076       414

3.14009661835748

              precision    recall  f1-score   support

       notes    0.00000   0.00000   0.00000       0.0
     surgery    0.00000   0.00000   0.00000     413.0

    accuracy                        0.00000     413.0
   macro avg    0.00000   0.00000   0.00000     413.0
weighted avg    0.00000   0.00000   0.00000     413.0

0.0
                                                                     precision    recall  f1-score   support

                                             bones/teeth/legs/neuro    0.00000   0.00000   0.00000       0.0
diseases/serious illnesses/allergy/nutrition/general/birth/children    0.00000   0.00000   0.00000       0.0
                                  intestine/uniary/skin/heart+blood    0.00000   0.00000   0.00000       0.0
                                                              notes    0.00000   0.00000   0.00000       0.0
                                                            surgery    0.00000   0.00000   0.00000     413.0

             

In [58]:
kf.get_n_splits(data_j1)

12

In [59]:
#jasmine1 
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data_j1):
    x_train = []
    y_train = []

    x_train, x_test = data_j1['transcription'][train_index], data_j1['transcription'][test_index]
    y_train, y_test = data_j1['medical_specialty'][train_index], data_j1['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
    lsvc.fit(Train_X_Tfidf,y_train) 
    
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())
    print(accuracy_score(y_test, predictions_nuSVM)*100)
    
print(total/count)
print(res)

                        precision    recall  f1-score   support

             Care Type    0.00000   0.00000   0.00000         0
     Internal Medicine    1.00000   0.39614   0.56747       414
    Medicine Diagnosis    0.00000   0.00000   0.00000         0
Treatments and Surgery    0.00000   0.00000   0.00000         0
               surgery    0.00000   0.00000   0.00000         0

              accuracy                        0.39614       414
             macro avg    0.20000   0.07923   0.11349       414
          weighted avg    1.00000   0.39614   0.56747       414

39.61352657004831
                   precision    recall  f1-score   support

Internal Medicine    0.00000   0.00000   0.00000     414.0
          surgery    0.00000   0.00000   0.00000       0.0

         accuracy                        0.00000     414.0
        macro avg    0.00000   0.00000   0.00000     414.0
     weighted avg    0.00000   0.00000   0.00000     414.0

0.0
                   precision    recall  f1

In [60]:
kf.get_n_splits(data_j2)

12

In [61]:
#jasmine2
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data_j2):
    x_train = []
    y_train = []

    x_train, x_test = data_j2['transcription'][train_index], data_j2['transcription'][test_index]
    y_train, y_test = data_j2['medical_specialty'][train_index], data_j2['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
    lsvc.fit(Train_X_Tfidf,y_train) 
    
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())
    print(accuracy_score(y_test, predictions_nuSVM)*100)

print(total/count)
print(res)

              precision    recall  f1-score   support

  Category 1    0.00000   0.00000   0.00000         0
  Category 2    1.00000   0.39855   0.56995       414
  Category 4    0.00000   0.00000   0.00000         0
     surgery    0.00000   0.00000   0.00000         0

    accuracy                        0.39855       414
   macro avg    0.25000   0.09964   0.14249       414
weighted avg    1.00000   0.39855   0.56995       414

39.85507246376812
              precision    recall  f1-score   support

  Category 2    0.00000   0.00000   0.00000     414.0
     surgery    0.00000   0.00000   0.00000       0.0

    accuracy                        0.00000     414.0
   macro avg    0.00000   0.00000   0.00000     414.0
weighted avg    0.00000   0.00000   0.00000     414.0

0.0
              precision    recall  f1-score   support

  Category 2    0.00000   0.00000   0.00000     414.0
     surgery    0.00000   0.00000   0.00000       0.0

    accuracy                        0.00000     414.

In [20]:
x_train, x_test = train_data_temp['transcription'], data['transcription'][temp_list]
y_train, y_test = train_data_temp['medical_specialty'], data['medical_specialty'][temp_list]
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)
lsvc.fit(Train_X_Tfidf,y_train)
predictions_lSVM = lsvc.predict(Test_X_Tfidf)
print(metrics.classification_report(predictions_lSVM, y_test.to_numpy(), digits=5))

                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary    0.07458   0.36667   0.12394        60
    Consult - History and Phy.    0.29773   0.47985   0.36746       273
             Discharge Summary    0.87500   0.10108   0.18123       277
          ENT - Otolaryngology    0.50000   0.15873   0.24096        63
              Gastroenterology    0.60135   0.26488   0.36777       336
              General Medicine    0.32787   0.24793   0.28235       242
         Hematology - Oncology    0.35714   0.05155   0.09009        97
                    Nephrology    0.40000   0.01493   0.02878       134
                     Neurology    0.57143   0.37004   0.44920       227
                  Neurosurgery    0.94444   0.04106   0.07870       414
       Obstetrics / Gynecology    0.37975   0.32967   0.35294        91
                 Ophthalmology    0.57143   0.05882   0.10667        68
                    Orthopedic    0.47670   0.38776   0.42765  

In [17]:
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data):
    # print("TRAIN:", len(train_index), "TEST:", len(test_index))
    # for train_index, test_index in kf.split(data):
    x_train = []
    y_train = []

    x_train, x_test = data['transcription'][train_index], data['transcription'][test_index]
    y_train, y_test = data['medical_specialty'][train_index], data['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
#     nsvc.fit(Train_X_Tfidf,y_train) 
    lsvc.fit(Train_X_Tfidf,y_train) 
    
#     predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())


    print(accuracy_score(y_test, predictions_nuSVM)*100)
#     total += accuracy_score(y_test, predictions_nuSVM)*100
#     count += 1

# print(count)
# print(total/count)
print(total/count)
print(res)
    


                             precision    recall  f1-score   support

       Allergy / Immunology    0.00000   0.00000   0.00000         0
                 Bariatrics    0.00000   0.00000   0.00000         0
 Cardiovascular / Pulmonary    0.00000   0.00000   0.00000         0
                  Dentistry    0.00000   0.00000   0.00000         0
           General Medicine    0.00000   0.00000   0.00000         0
                  Neurology    0.00000   0.00000   0.00000         0
                    Surgery    1.00000   0.57488   0.73006       414
                    Urology    0.00000   0.00000   0.00000         0

                   accuracy                        0.57488       414
                  macro avg    0.12500   0.07186   0.09126       414
               weighted avg    1.00000   0.57488   0.73006       414

57.48792270531401
              precision    recall  f1-score   support

     Surgery    1.00000   1.00000   1.00000       414

    accuracy                        1.000

In [18]:
x_train, x_test = data['transcription'][0:5], data['transcription'][0:5]
y_train, y_test = data['medical_specialty'][0:5], data['medical_specialty'][0:5]
                                                  
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)

nsvc.fit(Train_X_Tfidf,y_train)   
#classifier2.fit(Train_X_Tfidf,y_train)

predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
print(accuracy_score(predictions_nuSVM, y_test)*100)

100.0


In [ ]:
print(len(Tfidf_vect.vocabulary_))

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
print(Tfidf_vect.get_feature_names())